In [1]:
import os
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap as LSCm
from scipy.interpolate import griddata
from scipy.signal import savgol_filter
from scipy import integrate
from NesrHydrusAnalyst import *

In [3]:
src = '../Datasets/H3D2_SandDitch0011'

# Working with velocity

In [8]:
df= read_hydrus_data(folder=src, save_to_csv=False, read_velocities=True)
df.sample(3)

,n,x,y,z,Th_T0,Th_T5,Th_T15,Th_T30,Th_T60,Th_T120,...,V3_T240,V1_T500,V2_T500,V3_T500,V1_T720,V2_T720,V3_T720,V1_T1440,V2_T1440,V3_T1440
4404,4404,10.0515,13.7500,12.19070,0.20,0.200000,0.200000,0.200000,0.200000,0.200028,...,-0.000015,-0.000030,-0.000006,-0.000089,-9.619570e-05,-0.000007,-0.000076,-0.000080,-0.000002,0.000004
1944,1944,27.2782,18.4615,6.09483,0.15,0.230054,0.245116,0.265845,0.390000,0.372977,...,-0.000105,0.000027,-0.000002,-0.000068,-8.586860e-07,-0.000002,-0.000058,-0.000020,-0.000002,-0.000031
6545,6545,46.5295,6.2500,11.78960,0.20,0.200000,0.200000,0.200000,0.200006,0.204638,...,-0.000242,0.000014,0.000006,-0.000272,1.930540e-05,0.000005,-0.000132,0.000007,0.000001,-0.000049


In [9]:
print(list(df))

['n', 'x', 'y', 'z', 'Th_T0', 'Th_T5', 'Th_T15', 'Th_T30', 'Th_T60', 'Th_T120', 'Th_T180', 'Th_T240', 'Th_T500', 'Th_T720', 'Th_T1440', 'H_T0', 'H_T5', 'H_T15', 'H_T30', 'H_T60', 'H_T120', 'H_T180', 'H_T240', 'H_T500', 'H_T720', 'H_T1440', 'V1_T0', 'V2_T0', 'V3_T0', 'V1_T5', 'V2_T5', 'V3_T5', 'V1_T15', 'V2_T15', 'V3_T15', 'V1_T30', 'V2_T30', 'V3_T30', 'V1_T60', 'V2_T60', 'V3_T60', 'V1_T120', 'V2_T120', 'V3_T120', 'V1_T180', 'V2_T180', 'V3_T180', 'V1_T240', 'V2_T240', 'V3_T240', 'V1_T500', 'V2_T500', 'V3_T500', 'V1_T720', 'V2_T720', 'V3_T720', 'V1_T1440', 'V2_T1440', 'V3_T1440']


In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
n,8250.0,4.125500e+03,2381.714194,1.000000,2.063250e+03,4.125500e+03,6.187750e+03,8250.000000
x,8250.0,2.451462e+01,14.887410,-0.839105,1.155150e+01,2.439230e+01,3.752950e+01,49.960100
y,8250.0,9.983508e+00,6.314130,0.000000,4.615380e+00,9.991985e+00,1.538460e+01,20.000000
z,8250.0,1.186658e+01,6.483165,0.000000,6.190740e+00,1.178960e+01,1.778960e+01,22.981100
Th_T0,8250.0,1.778485e-01,0.034929,0.100000,1.500000e-01,2.000000e-01,2.000000e-01,0.200000
Th_T5,8250.0,1.886862e-01,0.066437,0.049132,1.999210e-01,2.000000e-01,2.000050e-01,0.430000
Th_T15,8250.0,2.104914e-01,0.086384,0.049792,2.000000e-01,2.000000e-01,2.063693e-01,0.430000
Th_T30,8250.0,2.355983e-01,0.097230,0.049960,2.000000e-01,2.000250e-01,2.886743e-01,0.430000
Th_T60,8250.0,2.954227e-01,0.100979,0.049914,2.000060e-01,2.919705e-01,3.900000e-01,0.430000
Th_T120,8250.0,2.915740e-01,0.075816,0.133659,2.084185e-01,3.117305e-01,3.615485e-01,0.426730


## The velocity components:
![From HYDRUS simulation](../Assets/H_velocity.png)
**V1 is the _x_ complonent of the velocity.**<br>
**V2 is the _y_ complonent of the velocity.**<br>
**V3 is the _z_ complonent of the velocity.**<br>


In [14]:
vr = 0
# v_mask = {0: 'Th', 1: 'H', 2.1:'V1', 2.2:'V2', 2.3:'V3'}[variable]
X, Z, M, x_vals, z_vals = get_grid_values(df, variable=vr)

# display(get_available_timesteps(df),
#         get_full_dimensions(data_frame),
#         get_legend_range(M.min(), M.max()),
#         'x_vals{}, z_vals{}, X{}, Z{}, M{}'.format(x_vals.shape, z_vals.shape,
#                                                    X.shape, Z.shape, M.shape))
X, Z, M, x_vals, z_vals

(array([[0.000, 0.500, 1.000, ..., 19.000, 19.500, 20.000],
        [0.000, 0.500, 1.000, ..., 19.000, 19.500, 20.000],
        [0.000, 0.500, 1.000, ..., 19.000, 19.500, 20.000],
        ...,
        [0.000, 0.500, 1.000, ..., 19.000, 19.500, 20.000],
        [0.000, 0.500, 1.000, ..., 19.000, 19.500, 20.000],
        [0.000, 0.500, 1.000, ..., 19.000, 19.500, 20.000]]),
 array([[0.000, 0.000, 0.000, ..., 0.000, 0.000, 0.000],
        [0.511, 0.511, 0.511, ..., 0.511, 0.511, 0.511],
        [1.021, 1.021, 1.021, ..., 1.021, 1.021, 1.021],
        ...,
        [21.960, 21.960, 21.960, ..., 21.960, 21.960, 21.960],
        [22.470, 22.470, 22.470, ..., 22.470, 22.470, 22.470],
        [22.981, 22.981, 22.981, ..., 22.981, 22.981, 22.981]]),
 array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [0.184, 0.184, 0.184, ..., 0.184, 0.184, 0.184],
        [nan, nan, nan, ..., nan, nan, nan],


In [15]:
get_available_timesteps(df)

IndexError: list index out of range